In [1]:
import pandas as pd

- `results.csv` joins `races.csv` to get `year`
- `results.csv` joins `drivers.csv` to get `driver`
- `results.csv` joins `constructors.csv` to get `constructor`
- `results.csv` joins `pitStops.csv` to get `duration`

In [39]:
results = pd.read_csv('data/results.csv')
results = results[['raceId', 'driverId', 'constructorId', 'position', 'rank']]

races = pd.read_csv('data/races.csv')
races.rename(columns={'name': 'raceName'}, inplace=True)
races = races[['raceId', 'year', 'raceName']]

drivers = pd.read_csv('data/drivers.csv')
drivers['driverFullname'] = drivers['forename'] + ' ' + drivers['surname']
drivers = drivers[['driverId', 'driverFullname']]
    
constructors = pd.read_csv('data/constructors.csv')
constructors.rename(columns={'name': 'constructorName'}, inplace=True)
constructors = constructors[['constructorId', 'constructorName']]

pitstops = pd.read_csv('data/pit_stops.csv')
pitstops = pitstops[['raceId', 'driverId', 'stop', 'duration', 'milliseconds']]

In [40]:
results = results.merge(races, on='raceId', how='inner')
results = results.merge(drivers, on='driverId', how='left')
results = results.merge(constructors, on='constructorId', how='left')
results = results.merge(pitstops, on=['raceId', 'driverId'], how='left')

In [44]:
columns = ['year', 'raceName', 'driverFullname', 'constructorName', 'position', 'stop', 'milliseconds']
times = results[columns].sort_values(by=['year'])\
                       .dropna(subset=['milliseconds'])

times.to_csv('data/times.csv', index=False)

In [45]:
times

,year,raceName,driverFullname,constructorName,position,stop,milliseconds
21544,2011,Italian Grand Prix,Jenson Button,McLaren,2,2.0,22095.0
21543,2011,Italian Grand Prix,Jenson Button,McLaren,2,1.0,21660.0
21542,2011,Italian Grand Prix,Sebastian Vettel,Red Bull,1,2.0,21378.0
21541,2011,Italian Grand Prix,Sebastian Vettel,Red Bull,1,1.0,22124.0
21532,2011,Belgian Grand Prix,Vitantonio Liuzzi,HRT,19,1.0,21766.0
...,...,...,...,...,...,...,...
30219,2022,Azerbaijan Grand Prix,Yuki Tsunoda,AlphaTauri,13,1.0,23083.0
30220,2022,Azerbaijan Grand Prix,Yuki Tsunoda,AlphaTauri,13,2.0,41500.0
30221,2022,Azerbaijan Grand Prix,Mick Schumacher,Haas F1 Team,14,1.0,26973.0
30249,2022,Canadian Grand Prix,Sebastian Vettel,Aston Martin,12,1.0,23406.0
